In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Set random seed for reproducibility
torch.manual_seed(42)

# Define the autoencoder architecture
class BinaryAutoencoder(nn.Module):
    def __init__(self, input_dim=8, latent_dim=1):
        super(BinaryAutoencoder, self).__init__()
        
        # Encoder: 8 binary digits -> 1 condensed representation
        self.encoder = nn.Sequential(
            # nn.Linear(input_dim, 6),
            # nn.ReLU(),
            # nn.Linear(6, 4),
            # nn.ReLU(),
            nn.Linear(input_dim, latent_dim),
            # nn.Linear(4, latent_dim)
        )
        
        # Decoder: 1 condensed representation -> 8 binary digits
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 2),
            # nn.ReLU(),
            torch.sin(),
            nn.Linear(2, 3),
            torch.sin(),
            # nn.ReLU(),
            nn.Linear(3, 4),
            torch.sin(),
            # nn.ReLU(),
            nn.Linear(4, 5),
            torch.sin(),
            # nn.ReLU(),
            nn.Linear(5, input_dim),
            nn.Sigmoid()  # Output between 0 and 1 for binary reconstruction
        )
    
    def forward(self, x):
        # Encode the input
        latent = self.encoder(x)

        latent_ints = []
        for inp in x:
            latent_int = np.array([sum(2 ** i for i, _ in enumerate(reversed(inp)) if _ == 1)])
            latent_ints.append(latent_int)

        latent_int = np.array(latent_ints)

        # Decode the latent representation
        reconstructed = self.decoder(torch.tensor(latent_int, dtype=torch.float32))
        return reconstructed, latent, latent_int

# Generate training data (binary representations of 0-255)
def generate_binary_data():
    data = []
    for i in range(256):
        # Convert to binary and pad to 8 digits
        binary = format(i, '08b')
        # Convert binary string to list of floats
        binary_list = [float(bit) for bit in binary]
        data.append(binary_list)
    return torch.tensor(data, dtype=torch.float32)

# Main training function
def train_autoencoder(epochs=1000, learning_rate=0.001):
    # Generate the dataset
    binary_data = generate_binary_data()
    
    # Create the model, loss function, and optimizer
    model = BinaryAutoencoder()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(epochs):
        # Forward pass
        reconstructed, latent, latent_int = model(binary_data)
        
        # Calculate loss
        loss = criterion(reconstructed, binary_data)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print progress every 100 epochs
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model

# Test the autoencoder
def test_autoencoder(model, num_to_test=10):
    binary_data = generate_binary_data()
    
    # Get reconstructions
    with torch.no_grad():
        reconstructed, latent, latent_int = model(binary_data)
    
    # Print results for a few examples
    print("\nOriginal -> Latent -> Reconstructed (rounded):")
    for i in range(num_to_test):
        original = binary_data[i].numpy()
        print(original)
        latent_val = latent[i].item()
        recon = reconstructed[i].numpy()
        print(recon)
        recon_rounded = np.round(recon).astype(int)
        
        # Convert binary arrays to integers for readability
        original_int = int(''.join(map(str, original.astype(int))), 2)
        recon_int = int(''.join(map(str, recon_rounded)), 2)
        
        print(f"Number: {original_int}, Binary: {original}, Latent: {latent_val:.4f}, Latent Int: {latent_int[i].item()}, Reconstructed: {recon_rounded}, As Int: {recon_int}")
    
    # Calculate accuracy
    rounded_reconstructed = torch.round(reconstructed)
    accuracy = (rounded_reconstructed == binary_data).float().mean().item() * 100
    print(f"\nBit-level accuracy: {accuracy:.2f}%")
    
    # Calculate perfect reconstructions (all 8 bits correct)
    perfect_reconstructions = torch.all(rounded_reconstructed == binary_data, dim=1).float().mean().item() * 100
    print(f"Perfect reconstruction rate: {perfect_reconstructions:.2f}%")

# Run the training and testing
if __name__ == "__main__":
    print("Training autoencoder...")
    trained_model = train_autoencoder(epochs=2000)
    
    print("\nTesting autoencoder...")
    test_autoencoder(trained_model, num_to_test=15)
    
    # Save model
    torch.save(trained_model.state_dict(), "binary_autoencoder.pth")
    print("\nModel saved to 'binary_autoencoder.pth'")

Training autoencoder...
Epoch [100/2000], Loss: 0.2513
Epoch [200/2000], Loss: 0.2460
Epoch [300/2000], Loss: 0.2406
Epoch [400/2000], Loss: 0.2347
Epoch [500/2000], Loss: 0.2296
Epoch [600/2000], Loss: 0.2236
Epoch [700/2000], Loss: 0.2183
Epoch [800/2000], Loss: 0.2142
Epoch [900/2000], Loss: 0.2112
Epoch [1000/2000], Loss: 0.2095
Epoch [1100/2000], Loss: 0.2083
Epoch [1200/2000], Loss: 0.2073
Epoch [1300/2000], Loss: 0.2062
Epoch [1400/2000], Loss: 0.2051
Epoch [1500/2000], Loss: 0.2039
Epoch [1600/2000], Loss: 0.2026
Epoch [1700/2000], Loss: 0.2009
Epoch [1800/2000], Loss: 0.1992
Epoch [1900/2000], Loss: 0.1975
Epoch [2000/2000], Loss: 0.1960

Testing autoencoder...

Original -> Latent -> Reconstructed (rounded):
[0. 0. 0. 0. 0. 0. 0. 0.]
[6.3811399e-32 1.8662856e-04 4.2624983e-05 3.6060226e-01 4.4595975e-01
 4.6022129e-01 4.8050037e-01 4.9308878e-01]
Number: 0, Binary: [0. 0. 0. 0. 0. 0. 0. 0.], Latent: 0.3117, Latent Int: 0, Reconstructed: [0 0 0 0 0 0 0 0], As Int: 0
[0. 0. 0. 0